In [1]:
import os
import glob
import random
import cv2
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
from torch.utils.data import DataLoader
from mpl_toolkits.axes_grid1 import ImageGrid
from torchvision.utils import make_grid
import torchvision.transforms as tt
import albumentations as A
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from PIL import Image
from albumentations.pytorch import ToTensorV2

In [ ]:
torch.cuda.is_available()

True

In [2]:
! pip install patchify
! pip install Pillow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from patchify import patchify, unpatchify


In [4]:
Image.MAX_IMAGE_PIXELS = 1000000000 


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
COLORMAP = [
        [0, 0, 0],
        [255, 255, 0],
        [150, 150, 150],
        [0, 255, 0],
        [0, 0, 255],
        [0, 150, 255] 
    ]

CLASSES = [
        'background',
        'yellow',
        'grey',
        'green',
        'blue',
        'red'
    ]


In [7]:
t1 = A.Compose([
    #A.Resize(160,240),
    #A.augmentations.transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
    ToTensorV2()
])

In [8]:
class encoding_block(nn.Module):
    def __init__(self,in_channels, out_channels):
        super(encoding_block,self).__init__()
        model = []
        model.append(nn.Conv2d(in_channels, out_channels, 3, 1, 1, bias=False))
        model.append(nn.BatchNorm2d(out_channels))
        model.append(nn.ReLU(inplace=True))
        model.append(nn.Conv2d(out_channels, out_channels, 3, 1, 1, bias=False))
        model.append(nn.BatchNorm2d(out_channels))
        model.append(nn.ReLU(inplace=True))
        self.conv = nn.Sequential(*model)
    def forward(self, x):
        return self.conv(x) 

In [9]:
class unet_model(nn.Module):
    def __init__(self,out_channels=6,features=[64, 128, 256, 512]):
        super(unet_model,self).__init__()
        self.pool = nn.MaxPool2d(kernel_size=(2,2),stride=(2,2))
        self.conv1 = encoding_block(3,features[0])
        self.conv2 = encoding_block(features[0],features[1])
        self.conv3 = encoding_block(features[1],features[2])
        self.conv4 = encoding_block(features[2],features[3])
        self.conv5 = encoding_block(features[3]*2,features[3])
        self.conv6 = encoding_block(features[3],features[2])
        self.conv7 = encoding_block(features[2],features[1])
        self.conv8 = encoding_block(features[1],features[0])        
        self.tconv1 = nn.ConvTranspose2d(features[-1]*2, features[-1], kernel_size=2, stride=2)
        self.tconv2 = nn.ConvTranspose2d(features[-1], features[-2], kernel_size=2, stride=2)
        self.tconv3 = nn.ConvTranspose2d(features[-2], features[-3], kernel_size=2, stride=2)
        self.tconv4 = nn.ConvTranspose2d(features[-3], features[-4], kernel_size=2, stride=2)        
        self.bottleneck = encoding_block(features[3],features[3]*2)
        self.final_layer = nn.Conv2d(features[0],out_channels,kernel_size=1)
    def forward(self,x):
        skip_connections = []
        x = self.conv1(x)
        skip_connections.append(x)
        x = self.pool(x)
        x = self.conv2(x)
        skip_connections.append(x)
        x = self.pool(x)
        x = self.conv3(x)
        skip_connections.append(x)
        x = self.pool(x)
        x = self.conv4(x)
        skip_connections.append(x)
        x = self.pool(x)
        x = self.bottleneck(x)
        skip_connections = skip_connections[::-1]
        x = self.tconv1(x)
        x = torch.cat((skip_connections[0], x), dim=1)
        x = self.conv5(x)
        x = self.tconv2(x)
        x = torch.cat((skip_connections[1], x), dim=1)
        x = self.conv6(x)
        x = self.tconv3(x)
        x = torch.cat((skip_connections[2], x), dim=1)
        x = self.conv7(x)        
        x = self.tconv4(x)
        x = torch.cat((skip_connections[3], x), dim=1)
        x = self.conv8(x)
        x = self.final_layer(x)
        return x

In [10]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
model = unet_model().to(DEVICE)
model=torch.load("/content/drive/MyDrive/biopseModel.pth", map_location=DEVICE)

model.eval()

unet_model(
  (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv1): encoding_block(
    (conv): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
    )
  )
  (conv2): encoding_block(
    (conv): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine

In [74]:
image_path="/content/drive/MyDrive/patchedData/images/patch_100.png"

orig_image = Image.open(image_path)  # for example (3456, 5184, 3)
orig_image = np.asarray(orig_image)

In [75]:
patches = patchify(orig_image, (512, 512, 3), step=512)


In [76]:
softmax = nn.Softmax(dim=1)


In [77]:
mask_patches = np.empty(patches.shape).astype(np.uint8)

In [78]:
patches.shape

(1, 1, 1, 512, 512, 3)

In [79]:
def process_mask( rgb_mask): 
         output_mask = []        
         for i, color in enumerate(COLORMAP):
             cmap = np.all(np.equal(rgb_mask, color), axis=-1)
             #print(np.equal(rgb_mask, color).shape)

             output_mask.append(cmap)
         #print(len(output_mask))
         output_mask = np.argmax(output_mask, axis=0)
         #print(output_mask.shape)
         
         #print(np.unique(output_mask))
         #print(output_mask)
         aug = t1(image=output_mask)
         mask = aug['image'].to(DEVICE)   
         return mask.unsqueeze(0) 

In [80]:
rgb_mask = Image.open("/content/drive/MyDrive/patchedData/masks/mask_patch_100.png") 
rgb_mask = np.asarray(rgb_mask)

#orig_mask=process_mask( mask_path)
target_patches = patchify(rgb_mask, (512, 512, 3), step=512)

#dice_scores=[]

In [81]:
dice_score = 0

for i in range(patches.shape[0]):
        for j in range(patches.shape[1]):
              patch = patches[i, j, 0]
              image = np.array(patch)
              image=image/255.0
              image = t1(image=image)["image"].to(DEVICE)
              image = image.unsqueeze(0) 

              prediction = torch.argmax(softmax(model(image.float())),axis=1)

              target=process_mask( target_patches[i,j,0]).to(DEVICE)
              
              dice_score +=(2 * (prediction * target).sum()) / ((prediction + target).sum() + 1e-8)
              #dice_scores.append((2 * (prediction * target).sum()) / ((prediction + target).sum() + 1e-8))

              prediction=prediction.to('cpu').detach()
              pred=np.array(prediction[0])
              pred3channel = np.zeros( ( pred.shape[0], pred.shape[1], 3 ), np.uint8 )
              
              r=pred3channel[:,:,0]
              g=pred3channel[:,:,1]
              b=pred3channel[:,:,2]

 
              #unique_values=np.unique(prediction[0])

              for c in range(0, len(COLORMAP)):
                 r[pred==c]=COLORMAP[c][0]
                 g[pred==c]=COLORMAP[c][1]
                 b[pred==c]=COLORMAP[c][2]

              #for r in range(np.array(prediction).shape[0]):
              #  for c in range(np.array(prediction).shape[1]):
              #           pred3channel[0,r,c] = image[0][0,r,c]
              #           pred3channel[1,r,c] = image[0][1,r,c]
              #           pred3channel[2,r,c] = image[0][2,r,c]

              pred3channel[:,:,0]=r
              pred3channel[:,:,1]=g
              pred3channel[:,:,2]=b

              mask_patches[i, j, 0] = pred3channel


In [82]:
dice_score.detach().cpu().numpy()

array(1.6060283, dtype=float32)

In [83]:
dice_score.detach().cpu().numpy().astype(int)/(patches.shape[0]* patches.shape[1])

1.0

In [72]:
image_height, image_width, channel_count = orig_image.shape
patch_height, patch_width, step = 512, 512, 512
patch_shape = (patch_height, patch_width, channel_count)


In [73]:
output_height = image_height - (image_height - patch_height) % step
output_width = image_width - (image_width - patch_width) % step
output_shape = (output_height, output_width, channel_count)
output_image = unpatchify(mask_patches, output_shape)
output_image = Image.fromarray(output_image)
output_image.save("/content/drive/MyDrive/patchedData/predict-3/mask_patch_100.png")